## **Download Dataset**

Dataset bisa di download [disini](https://drive.google.com/file/d/1x0NiGnZap4MMvyPsqMnVP7xKoInzV1IE/view?usp=share_link)

Setelah mengunduh dataset yang dibutuhkan dataset akan dimasukkan secara manual ke dalam google collab untuk selanjutnya diproses ke dalam model rekomendasi sistem.

## **Import Library**
Import library digunakan untuk menganalisa dataset 

In [ ]:
import re
import random
import nltk
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

## **Download NLTK Package**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv("/content/dataset_rs.csv")
df.head()

,nama,vote_average,vote_count,type,htm_weekday,htm_weekend,latitude,Address,longitude,description
0,Candi Borobudur,"4,7",81922,Budaya dan Sejarah,50000,50000,"-7,607086854","Jl. Badrawati, Borobudur, Magelang, Jawa Tengah","110,2036226",Candi yang pernah masuk sebagai salah satu dar...
1,Candi Prambanan,"4,7",71751,Budaya dan Sejarah,50000,50000,"-7,751834561","Jl. Raya Solo – Yogyakarta No.16, Kranggan, Bo...","110,4915318",Candi Prambanan adalah kompleks candi Hindu te...
2,Tebing Breksi,"4,4",51431,Alam,10000,10000,"-7,781476547","Desa Sambirejo, Kecamatan Prambanan, Kabupaten...","110,5045757",Tebing Breksi merupakan tempat wisata yang ber...
3,Gembira Loka Zoo,"4,5",36337,Buatan,20000,25000,"-7,8062344","Jl. Kebun Raya No. 2, Rejowinangun, Kotagede, ...","110,3967977",Gambira Loka adalah kebun binatang yang berada...
4,The Palace of Yogyakarta (Keraton Yogyakarta),"4,6",30091,Budaya dan Sejarah,8000,8000,"-7,8052845","Jl. Rotowijayan Blok No. 1, Panembahan, Kraton...","110,3642031",Kompleks keraton merupakan museum yang menyimp...


In [ ]:
df.describe()

,vote_count
count,297.000000
mean,2930.484848
std,8123.339376
min,1.000000
25%,107.000000
50%,504.000000
75%,2245.000000
max,81922.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nama          297 non-null    object
 1   vote_average  297 non-null    object
 2   vote_count    297 non-null    int64 
 3   type          296 non-null    object
 4   htm_weekday   291 non-null    object
 5   htm_weekend   291 non-null    object
 6   latitude      297 non-null    object
 7   Address       296 non-null    object
 8   longitude     297 non-null    object
 9   description   297 non-null    object
dtypes: int64(1), object(9)
memory usage: 23.3+ KB


#**Deskripsi Wisata**
Deskripsi wisata berguna untuk mengetahui isi informasi dari masing — masing wisata yang ada pada dataset yang kita masukkan.

In [ ]:
def print_description(index):
    example = df[df.index == index][['nama', 'Address', 'description']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Alamat:', example[1])
        print('Deskripsi:', example[2])    

In [ ]:
print_description(1)

Candi Prambanan
Alamat: Jl. Raya Solo – Yogyakarta No.16, Kranggan, Bokoharjo, Kec. Prambanan, Kabupaten Sleman, Daerah Istimewa Yogyakarta
Deskripsi: Candi Prambanan adalah kompleks candi Hindu terbesar di Indonesia yang dibangun pada abad ke-9 masehi. Candi ini dipersembahkan untuk Trimurti, tiga dewa utama Hindu yaitu Brahma sebagai dewa pencipta, Wisnu sebagai dewa pemelihara, dan Siwa sebagai dewa pemusnah. Berdasarkan prasasti Siwagrha nama asli kompleks candi ini adalah Siwagrha (bahasa Sanskerta yang bermakna 'Rumah Siwa'), dan memang di garbagriha (ruang utama) candi ini bersemayam arca Siwa Mahadewa setinggi tiga meter yang menujukkan bahwa di candi ini dewa Siwa lebih diutamakan.


#**Text Preprocessing**
Setelah melakukan analisa dan deskripsi wisata, proses selanjutnya melakukan text preprocessing yang bertujuan agar data yang dipakai nanti dapat diproses menjadi angka dengan TF-IDF dan cosine similarity nanti. Data yang akan dipakai hanya kolom ‘description’ saja karena agar bisa mendapatkan kesamaan pada pengaplikasiannya nanti.

In [ ]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopwords = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopwords) # hapus stopword dari kolom nama
    return text
    
df['nama_clean'] = df['nama'].apply(clean_text)
df

,nama,vote_average,vote_count,type,htm_weekday,htm_weekend,latitude,Address,longitude,description,nama_clean
0,Candi Borobudur,"4,7",81922,Budaya dan Sejarah,50000,50000,"-7,607086854","Jl. Badrawati, Borobudur, Magelang, Jawa Tengah","110,2036226",Candi yang pernah masuk sebagai salah satu dar...,candi borobudur
1,Candi Prambanan,"4,7",71751,Budaya dan Sejarah,50000,50000,"-7,751834561","Jl. Raya Solo – Yogyakarta No.16, Kranggan, Bo...","110,4915318",Candi Prambanan adalah kompleks candi Hindu te...,candi prambanan
2,Tebing Breksi,"4,4",51431,Alam,10000,10000,"-7,781476547","Desa Sambirejo, Kecamatan Prambanan, Kabupaten...","110,5045757",Tebing Breksi merupakan tempat wisata yang ber...,tebing breksi
3,Gembira Loka Zoo,"4,5",36337,Buatan,20000,25000,"-7,8062344","Jl. Kebun Raya No. 2, Rejowinangun, Kotagede, ...","110,3967977",Gambira Loka adalah kebun binatang yang berada...,gembira loka zoo
4,The Palace of Yogyakarta (Keraton Yogyakarta),"4,6",30091,Budaya dan Sejarah,8000,8000,"-7,8052845","Jl. Rotowijayan Blok No. 1, Panembahan, Kraton...","110,3642031",Kompleks keraton merupakan museum yang menyimp...,palace yogyakarta keraton yogyakarta
...,...,...,...,...,...,...,...,...,...,...,...
292,Pantai Pasir Mendit,3,1,Pantai,5000,5000,"-7,8943492","Dusun Pasir Mendit, Desa Jangkaran, Kecamatan ...","110,0194606",Menyediakan pemandangan dan wisata air yang me...,pantai pasir mendit
293,Pemancingan Adi Winata,5,1,"Wisata, Kuliner",0,0,"-7,7045773","Kabupaten Sleman, Yogyakarta","110,5120115",Menyediakan tempat pemancingan dan resto yang ...,pemancingan adi winata
294,Taman Edukasi dan Outbound Sunan Kalijaga,5,1,"Taman Bermain, Edukasi Wisata, Buatan",0,0,"-7,8092066","Jl. Jomblangan, Sorowajan, Banguntapan, Kec. B...","110,4132518",Wahana wisata edukasi keluarga dan komunitas d...,taman edukasi dan outbound sunan kalijaga
295,Wisata Air Wanatirta Kencana,5,1,Wisata Air,10000,15000,"-7,8914433",Jalan Jogja-Wonosari Km 30 tepatnya disebelah ...,"110,548354",Wahana bermain air di Taman Wisata Air Tirta K...,wisata air wanatirta kencana


In [ ]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['nama_clean', 'Address', 'description']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Alamat:', example[1])
        print('Deskripso:', example[2])

In [ ]:
print_description_clean(0)

candi borobudur
Alamat: Jl. Badrawati, Borobudur, Magelang, Jawa Tengah
Deskripso: Candi yang pernah masuk sebagai salah satu dari Tujuh Keajaiban Dunia ini dihiasi oleh banyak relief di dinding candi. Relief yang ada di Candi Borobudur mencapai 2.672 dan panjangnya bisa mencapai 6 kilometer


#**TF-IDF dan Cosine Similarity**
Setelah data di preprocessing, gunakan library TF-IDF dan cosine similarity agar bisa diubah menjadi angka berupa matriks. (Untuk mengetahuinya lebih lanjut

In [ ]:
df.set_index('nama_clean', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['nama'])

In [ ]:
similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
similarity

array([[1.        , 0.14260218, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.14260218, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
# Set index utama di kolom 'nama'
indices = pd.Series(df.index)
indices[:10]

0                           candi borobudur
1                           candi prambanan
2                             tebing breksi
3                          gembira loka zoo
4      palace yogyakarta keraton yogyakarta
5                                taman sari
6               hutan pinus mangunan dlingo
7                                 jogja bay
8    world landmarks merapi park yogyakarta
9                         lost world castle
Name: nama_clean, dtype: object

#**Modelling**
Pada tahap permodelan, saya membuat fungsi untuk rekomendasi wisata yang sama sesuai hasil dari TF-IDF dan cosine similarity yang dibuat. Hasil yang akan ditampilkan berupa 10 urutan terdekat dengan nama wisata yang kita definisikan.

In [ ]:
def recommendations(nama, similarity = similarity):
    recommended_wisata = []
    # Mengambil nama wisata berdasarkan variabel indicies
    idx = indices[indices == nama].index[0]
    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(similarity[idx]).sort_values(ascending = False)
    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_wisata.append(list(df.index)[i])
        
    return recommended_wisata

In [ ]:
recommendations('taman sari')

['candi sari',
 'pakem sari water park',
 'taman lampion taman pelangi',
 'taman glugut',
 'taman nggirli',
 'taman wisata kaliurang',
 'taman pelangi jogja',
 'taman pintar yogyakarta',
 'taman air menari',
 'waterbyur taman air']